# **DESAFÍO Nº6: REGRESIÓN LINEAL**
### **Módulo 4: Análisis Estadístico con Python**
#### *Carrera Data Science - G-93 - Desafio Latam*
#### Estudiante: Felipe Leal Arancibia

En este desafío validaremos nuestros conocimientos aprendidos de regresión lineal. Para lograrlo, necesitarás aplicar los modelos aprendidos en clase, utilizando de apoyo el archivo fish.csv.

Lee todo el documento antes de comenzar el desarrollo individual, para asegurarte de tener el máximo de puntaje y enfocar bien los esfuerzos.

Tiempo asociado: 2 horas cronológicas

## Descripción

#### Este conjunto de datos es un registro de las ventas en el mercado de 7 especies comunes de peces diferentes. Con este conjunto de datos, se busca realizar un modelo predictivo utilizando datos adecuados para máquinas y estimar el peso de los peces basándose en la longitud y ancho del pez.

#### El conjunto de datos incluye información sobre varias dimensiones del pez, que se utilizan para predecir su peso. Cada fila en el conjunto de datos representa una entrada de un pez individual y contiene varias columnas con información relevante. Las dimensiones del pez y otros atributos en el conjunto de datos podrían incluir:

##### - Species: La especie del pez.
##### - Weight: El peso del pez, que es la variable que se intenta predecir.
##### - Length1, Length2, Length3: Diferentes longitudes del pez.
##### - Height: Altura del pez.
##### - Width: Ancho del pez.


#### Los datos en este archivo permiten realizar análisis estadísticos y modelado predictivo para determinar cómo las dimensiones del pez están relacionadas con su peso. Esto puede ser útil en la industria pesquera y en la investigación científica para comprender mejor las relaciones entre las características físicas de los peces y su peso. Para esto se te solicita:

In [1]:
# Primero importar todas las potenciales librería sa utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats


#### 1. Crea una nueva columna que incluya el volumen de pez, asumiéndolo como un cilindro. Para un cilindro con una base circular, el área de la base (A) es igual a πr2, donde "r" es el radio de la base del cilindro, y la altura (h) es la distancia entre las dos bases circulares.

#### 2. Verifica la correlación posible entre las variables, numérica y gráficamente. ¿Aumenta la correlación al añadir el volumen? Explica.

#### 3. Construye un modelo de regresión lineal que relacione el volumen y el peso de los peces. Realiza una breve evaluación del modelo y grafícalo.

#### 4. ¿Qué sucede si aplicas el modelo anterior por separado a cada especie de peces? Explica.



#### 5. Construye un modelo de regresión que relacione dos o más variables con el peso de los peces. Realiza una breve evaluación del modelo.

In [5]:
# 1. Considerando las variables earn, height y age, vamos a suponer que nuestro dataset fuera una población completa. (Prepara los datos adecuadamente).

# Cargamos el dataset
df_earnings = pd.read_csv('earnings.csv')

df_earnings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1816 entries, 0 to 1815
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   height            1816 non-null   int64  
 1   weight            1789 non-null   float64
 2   male              1816 non-null   int64  
 3   earn              1816 non-null   float64
 4   earnk             1816 non-null   float64
 5   ethnicity         1816 non-null   object 
 6   education         1814 non-null   float64
 7   mother_education  1572 non-null   float64
 8   father_education  1521 non-null   float64
 9   walk              1816 non-null   int64  
 10  exercise          1816 non-null   int64  
 11  smokenow          1815 non-null   float64
 12  tense             1815 non-null   float64
 13  angry             1815 non-null   float64
 14  age               1816 non-null   int64  
dtypes: float64(9), int64(5), object(1)
memory usage: 212.9+ KB


-
-
-
___0___o___
-
-
-

-
-
-
___0___o___
-
-
-

-
-
-
___0_FiN__o___
-
-
-

### Requerimientos

1. Analiza correlaciones entre variables de un dataset, en forma numérica y gráfica. (2 Puntos)
2. Construye modelos de relación lineal en Python. (5 Puntos)
3. Evalúa modelos de regresión lineal. (3 Puntos)

¡Mucho éxito!

Consideraciones y recomendaciones

Considera que el pez es un cilindro por tanto el volumen de este será:
Volumen = Área de la base × Altura
Para un cilindro con una base circular, el área de la base (A) es igual a πr2, donde "r" es el radio de la base del cilindro, y la altura (h) es la distancia entre las dos bases circulares.

Por lo tanto, la ecuación general para el volumen (V) de un cilindro es:

np.pi*df['Height']*(df['Width']/2)**2